In [1]:
import os, random

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn

from MyPyTorch import *

os.makedirs('model_pytorch', exist_ok=True)
os.makedirs('figure', exist_ok=True)

In [2]:
print(torch.__version__)
print(torch.version.cuda)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name())

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

1.10.1+cu113
11.3
True
Quadro P2000


In [3]:
def getData_ScoreRangePredict(course_name = None):
    x_data = pd.read_csv('data/train/ScorePredict_x_{}.csv'.format(course_name) if course_name else 'data/train/ScorePredict_x.csv')
    y_data = pd.read_csv('data/train/ScorePredict_y_{}.csv'.format(course_name) if course_name else 'data/train/ScorePredict_y.csv')
    n_data = pd.read_csv('data/train/normalize.csv')
    n_data.set_index('col', inplace=True)
    print('Num of features:', len(n_data))

    x_train = x_data[x_data['group'] == 'training']
    x_valid = x_data[x_data['group'] == 'validation']
    x_test = x_data[x_data['group'] == 'testing']
    y_train = y_data[y_data['group'] == 'training']
    y_valid = y_data[y_data['group'] == 'validation']
    y_test = y_data[y_data['group'] == 'testing']

    x_train = n_data.apply(lambda n: normalize(n, x_train), axis = 1).T
    x_valid = n_data.apply(lambda n: normalize(n, x_valid), axis = 1).T
    x_test = n_data.apply(lambda n: normalize(n, x_test), axis = 1).T
    y_train.drop(['group'], axis = 1, inplace = True)
    y_valid.drop(['group'], axis = 1, inplace = True)
    y_test.drop(['group'], axis = 1, inplace = True)

    x_train, x_valid, x_test, y_train, y_valid, y_test = map(lambda x: torch.tensor(x.to_numpy(), dtype=torch.float), (x_train, x_valid, x_test, y_train, y_valid, y_test))
    return (x_train, x_valid, y_train, y_valid)

def getData_ScorePredict(course_name = None, file_model='model_pytorch/ScoreRangePredict.pt'):
    x_data = pd.read_csv('data/train/ScorePredict_x_{}.csv'.format(course_name) if course_name else 'data/train/ScorePredict_x.csv')
    y_data = pd.read_csv('data/train/ScorePredict_y_{}.csv'.format(course_name) if course_name else 'data/train/ScorePredict_y.csv')
    z_data = pd.read_csv('data/train/ScorePredict_z_{}.csv'.format(course_name) if course_name else 'data/train/ScorePredict_z.csv')
    n_data = pd.read_csv('data/train/normalize.csv')
    n_data.set_index('col', inplace=True)
    print('Num of features:', len(n_data))

    x_train = x_data[x_data['group'] == 'training']
    x_valid = x_data[x_data['group'] == 'validation']
    x_test  = x_data[x_data['group'] == 'testing']
    y_train = y_data[y_data['group'] == 'training']
    y_valid = y_data[y_data['group'] == 'validation']
    y_test  = y_data[y_data['group'] == 'testing']
    z_train = z_data[z_data['group'] == 'training']
    z_valid = z_data[z_data['group'] == 'validation']
    z_test  = z_data[z_data['group'] == 'testing']

    x_train = n_data.apply(lambda n: normalize(n, x_train), axis = 1).T
    x_valid = n_data.apply(lambda n: normalize(n, x_valid), axis = 1).T
    x_test = n_data.apply(lambda n: normalize(n, x_test), axis = 1).T
    y_train.drop(['group'], axis = 1, inplace = True)
    y_valid.drop(['group'], axis = 1, inplace = True)
    y_test.drop(['group'], axis = 1, inplace = True)
    z_train.drop(['group'], axis = 1, inplace = True)
    z_valid.drop(['group'], axis = 1, inplace = True)
    z_test.drop(['group'], axis = 1, inplace = True)
    x_train, x_valid, x_test, y_train, y_valid, y_test, z_train, z_valid, z_test = \
        map(lambda x: torch.tensor(x.to_numpy(), dtype=torch.float), (x_train, x_valid, x_test, y_train, y_valid, y_test, z_train, z_valid, z_test))

    model_srp = torch.load(file_model)
    index_train = (torch.argmax(model_srp(x_train), dim=1) == torch.argmax(y_train, dim=1))
    x_train = x_train[index_train]
    z_train = z_train[index_train]
    index_valid = (torch.argmax(model_srp(x_valid), dim=1) == torch.argmax(y_valid, dim=1))
    x_valid = x_valid[index_valid]
    z_valid = z_valid[index_valid]
    index_test = (torch.argmax(model_srp(x_test), dim=1) == torch.argmax(y_test, dim=1))
    x_test = x_test[index_test]
    z_test = z_test[index_test]

    return (x_train, x_valid, x_test, z_train, z_valid, z_test)
    # return (x_train, x_valid, z_train, z_valid)

In [4]:
x_train, x_valid, y_train, y_valid = getData_ScoreRangePredict()
xy_train = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), shuffle=True, batch_size=128, pin_memory=True)
x_train = x_train.to(device)
x_valid = x_valid.to(device)
y_train = y_train.to(device)
y_valid = y_valid.to(device)

model = ScoreRangePredict()
model = model.to(device)
criterion = torch.nn.MSELoss(reduction='mean')
criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters())

maxVA = 0
maxVA_ep = 0

for e in range(1000):
    ep = e + 1
    for xb, yb in xy_train:
        pred = model(xb.to(device))
        loss = criterion(pred, yb.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss = float(criterion(model(x_train), y_train))
    train_accu = float(accuracy(model(x_train), y_train))
    valid_loss = float(criterion(model(x_valid), y_valid))
    valid_accu = float(accuracy(model(x_valid), y_valid))
    save = ''
    if (maxVA < valid_accu) | (maxVA_ep < 1):
        maxVA = valid_accu
        maxVA_ep = ep
        save = '< save'
        torch.save(model, 'model_pytorch/ScoreRangePredict.pt')
    print('{:<4d}'.format(ep), 
        'train_loss', '{:.4f}'.format(train_loss), 
        'train_accu', '{:.4f}'.format(train_accu), 
        'valid_loss', '{:.4f}'.format(valid_loss), 
        'valid_accu', '{:.4f}'.format(valid_accu), 
        save)
    ## early drop
    if ep < 200: ## 至少執行200次
        pass
    elif maxVA_ep < ep/2: ## 連續10次小於maxValAcc的一半
        print('Accuracy of validation is CRASH !!')
        break
print('Training done, save model at VA =', maxVA)
## 轉換成CPU版本後儲存
model = torch.load('model_pytorch/ScoreRangePredict.pt')
model = model.to(torch.device('cpu'))
torch.save(model, 'model_pytorch/ScoreRangePredict.pt')

Num of features: 60


C:\Users\Administrator\Anaconda3\envs\tensorflow\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


1    train_loss 0.0222 train_accu 0.1752 valid_loss 0.0223 valid_accu 0.1786 < save
2    train_loss 0.0211 train_accu 0.2103 valid_loss 0.0212 valid_accu 0.2050 < save
3    train_loss 0.0207 train_accu 0.2239 valid_loss 0.0209 valid_accu 0.2205 < save
4    train_loss 0.0206 train_accu 0.2242 valid_loss 0.0208 valid_accu 0.2182 
5    train_loss 0.0200 train_accu 0.2546 valid_loss 0.0203 valid_accu 0.2480 < save
6    train_loss 0.0198 train_accu 0.2622 valid_loss 0.0201 valid_accu 0.2486 < save
7    train_loss 0.0197 train_accu 0.2633 valid_loss 0.0201 valid_accu 0.2532 < save
8    train_loss 0.0191 train_accu 0.2946 valid_loss 0.0196 valid_accu 0.2761 < save
9    train_loss 0.0187 train_accu 0.3109 valid_loss 0.0194 valid_accu 0.2861 < save
10   train_loss 0.0184 train_accu 0.3213 valid_loss 0.0191 valid_accu 0.2914 < save
11   train_loss 0.0182 train_accu 0.3325 valid_loss 0.0191 valid_accu 0.2976 < save
12   train_loss 0.0173 train_accu 0.3693 valid_loss 0.0184 valid_accu 0.3220 < sav

In [5]:
x_train, x_valid, x_test, z_train, z_valid, z_test= getData_ScorePredict()
xz_train = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, z_train), shuffle=True, batch_size=128, pin_memory=True)
x_train = x_train.to(device)
x_valid = x_valid.to(device)
x_test = x_test.to(device)
z_train = z_train.to(device)
z_valid = z_valid.to(device)
z_test = z_test.to(device)

model = Range2Score()
torch.save(model, 'model_pytorch/Range2Score.pt')
## 將分數區間轉換為分數
model = ScorePredict(srp_requires_grad = False)
model = model.to(device)
criterion = torch.nn.MSELoss(reduction='mean')

criterion = criterion.to(device)
optimizer = torch.optim.Adam(model.parameters())

minVL = 0
minVL_ep = 0

for e in range(1000):
    ep = e + 1
    for xb, zb in xz_train:
        pred = model(xb.to(device))
        loss = criterion(pred, zb.to(device))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_loss = float(criterion(model(x_train), z_train))
    valid_loss = float(criterion(model(x_valid), z_valid))
    test_loss = float(criterion(model(x_test), z_test))

    save = ''
    if (minVL > valid_loss) | (minVL_ep < 1):
        minVL = valid_loss
        minVL_ep = ep
        save = '< save'
        torch.save(model, 'model_pytorch/ScorePredict.pt')
    print('{:<4d}'.format(ep), 
        'train_loss', '{:.4f}'.format(train_loss), 
        # 'TA', '{:.4f}'.format(ta), 
        'valid_loss', '{:.4f}'.format(valid_loss), 
        # 'TA', '{:.4f}'.format(va), 
        'test_loss', '{:.4f}'.format(test_loss), 
        save)
    ## early drop
    if ep < 200: ## 至少執行200次
        pass
    elif minVL_ep < ep/2: ## 連續10次小於maxValAcc的一半
        print('Accuracy of validation is CRASH !!')
        break
print('Training done, save model at VL =', minVL)
## 轉換成CPU版本後儲存
model = torch.load('model_pytorch/ScorePredict.pt')
sample = random.sample([x for x in range(list(x_test.size())[1])], k=20)
print(torch.cat((model(x_test), z_test), 1)[sample])
model = model.to(torch.device('cpu'))
torch.save(model, 'model_pytorch/ScorePredict.pt')
model.save_r2s()

Num of features: 60
1    train_loss 182.5063 valid_loss 185.6148 test_loss 199.9757 < save
2    train_loss 10.8714 valid_loss 10.8745 test_loss 11.1119 < save
3    train_loss 9.0742 valid_loss 9.1867 test_loss 9.7028 < save
4    train_loss 7.8191 valid_loss 7.9959 test_loss 8.8125 < save
5    train_loss 6.7983 valid_loss 7.0788 test_loss 7.9163 < save
6    train_loss 6.0536 valid_loss 6.3901 test_loss 7.2564 < save
7    train_loss 5.5932 valid_loss 6.0534 test_loss 6.8832 < save
8    train_loss 5.3254 valid_loss 5.8600 test_loss 6.6490 < save
9    train_loss 5.1911 valid_loss 5.7846 test_loss 6.5417 < save
10   train_loss 4.9925 valid_loss 5.6235 test_loss 6.3998 < save
11   train_loss 5.0363 valid_loss 5.7199 test_loss 6.4548 
12   train_loss 4.8021 valid_loss 5.5037 test_loss 6.3035 < save
13   train_loss 4.7890 valid_loss 5.4788 test_loss 6.2897 < save
14   train_loss 5.0381 valid_loss 5.7403 test_loss 6.5358 
15   train_loss 4.6352 valid_loss 5.4044 test_loss 6.1445 < save
16   tra

In [6]:
## 針對各課程設計優化模型
course = pd.read_excel('data/other/course_list.xlsx', 'course')
course = course[course['for_train'] == 1]['course_name'].drop_duplicates()
for c in course:
    c_model = torch.load('model_pytorch/ScoreRangePredict.pt')

    torch.save(c_model, 'model_pytorch/ScoreRangePredict_{}.pt'.format(c))
    criterion = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(c_model.parameters())

    cx_t, cx_v, cy_t, cy_v = getData_ScoreRangePredict(c)
    train_dl = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(cx_t, cy_t), shuffle=True, batch_size=64) 
    maxVA = float(accuracy(c_model(cx_v), cy_v))
    maxVA_0 = maxVA
    maxVA_ep = 0

    print('Now processing model:', c)
    print('original VL:', maxVA)
    
    for e in range(1000):
        ep = e + 1
        for xb, yb in train_dl:
            pred = c_model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = float(criterion(c_model(cx_t), cy_t))
        train_accu = float(accuracy(c_model(cx_t), cy_t))
        valid_loss = float(criterion(c_model(cx_v), cy_v))
        valid_accu = float(accuracy(c_model(cx_v), cy_v))
        save = ''
        if maxVA < valid_accu:
            maxVA = valid_accu
            maxVA_ep = ep
            save = '< save'
            torch.save(c_model, 'model_pytorch/ScoreRangePredict_{}.pt'.format(c))
        print(c, '|', 
            '{:<4d}'.format(ep), 
            'train_loss', '{:.4f}'.format(train_loss), 
            'train_accu', '{:.4f}'.format(train_accu), 
            'valid_loss', '{:.4f}'.format(valid_loss), 
            'valid_accu', '{:.4f}'.format(valid_accu), 
            save)
        ## early drop
        if ep < 200: ## 至少執行200次
            pass
        elif maxVA_ep < ep/2: ## 連續10次小於maxValAcc的一半
            print('Accuracy of validation is CRASH !!')
            break
    print('ReTraining done, accuracy:', maxVA_0, '=>', maxVA)
    print('decrease:',maxVA_0 - maxVA, '\n')
    pass

Num of features: 60
Now processing model: 企業倫理
original VL: 0.8147208094596863
企業倫理 | 1    train_loss 0.0028 train_accu 0.9086 valid_loss 0.0070 valid_accu 0.7931 
企業倫理 | 2    train_loss 0.0024 train_accu 0.9230 valid_loss 0.0066 valid_accu 0.8046 
企業倫理 | 3    train_loss 0.0025 train_accu 0.9172 valid_loss 0.0068 valid_accu 0.7925 
企業倫理 | 4    train_loss 0.0026 train_accu 0.9149 valid_loss 0.0069 valid_accu 0.7887 
企業倫理 | 5    train_loss 0.0024 train_accu 0.9210 valid_loss 0.0062 valid_accu 0.8090 
企業倫理 | 6    train_loss 0.0026 train_accu 0.9141 valid_loss 0.0070 valid_accu 0.7881 
企業倫理 | 7    train_loss 0.0024 train_accu 0.9202 valid_loss 0.0065 valid_accu 0.8071 
企業倫理 | 8    train_loss 0.0024 train_accu 0.9216 valid_loss 0.0062 valid_accu 0.8166 < save
企業倫理 | 9    train_loss 0.0022 train_accu 0.9263 valid_loss 0.0061 valid_accu 0.8211 < save
企業倫理 | 10   train_loss 0.0024 train_accu 0.9228 valid_loss 0.0063 valid_accu 0.8154 
企業倫理 | 11   train_loss 0.0023 train_accu 0.9232 valid_loss 

In [7]:
## 使用MOE結合標準模型與優化模型
course = pd.read_excel('data/other/course_list.xlsx', 'course')
course = course[course['for_train'] == 1]['course_name'].drop_duplicates()
for c in course:
    c_model = ScoreRangePredict_MOE('model_pytorch/ScoreRangePredict_{}.pt'.format(c))

    criterion = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(c_model.parameters())

    cx_t, cx_v, cy_t, cy_v = getData_ScoreRangePredict(c)

    train_dl = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(cx_t, cy_t), shuffle=True, batch_size=64) 
    maxVA = float(accuracy(c_model(cx_v), cy_v))
    maxVA_0 = maxVA
    maxVA_ep = 0

    print('Now processing model:', c)
    print('original VA:', maxVA)
    
    for e in range(1000):
        ep = e + 1
        for xb, yb in train_dl:
            pred = c_model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = float(criterion(c_model(cx_t), cy_t))
        train_accu = float(accuracy(c_model(cx_t), cy_t))
        valid_loss = float(criterion(c_model(cx_v), cy_v))
        valid_accu = float(accuracy(c_model(cx_v), cy_v))
        save = ''
        if maxVA < valid_accu:
            maxVA = valid_accu
            maxVA_ep = ep
            save = '< save'
            torch.save(c_model, 'model_pytorch/ScoreRangePredict_MOE_{}.pt'.format(c))
        print(c, '|', 
            '{:<4d}'.format(ep), 
            'train_loss', '{:.4f}'.format(train_loss), 
            'train_accu', '{:.4f}'.format(train_accu), 
            'valid_loss', '{:.4f}'.format(valid_loss), 
            'valid_accu', '{:.4f}'.format(valid_accu), 
            save)
        ## early drop
        if ep < 200: ## 至少執行200次
            pass
        elif maxVA_ep < ep/2: ## 連續10次小於maxValAcc的一半
            print('Accuracy of validation is CRASH !!')
            break
    print('ReTraining done, maxVA:', maxVA_0, '=>', maxVA)
    pass

Num of features: 60
Now processing model: 企業倫理
original VA: 0.8229695558547974
企業倫理 | 1    train_loss 0.0021 train_accu 0.9312 valid_loss 0.0053 valid_accu 0.8261 < save
企業倫理 | 2    train_loss 0.0020 train_accu 0.9320 valid_loss 0.0053 valid_accu 0.8331 < save
企業倫理 | 3    train_loss 0.0020 train_accu 0.9332 valid_loss 0.0053 valid_accu 0.8350 < save
企業倫理 | 4    train_loss 0.0020 train_accu 0.9334 valid_loss 0.0053 valid_accu 0.8363 < save
企業倫理 | 5    train_loss 0.0020 train_accu 0.9334 valid_loss 0.0053 valid_accu 0.8363 
企業倫理 | 6    train_loss 0.0020 train_accu 0.9334 valid_loss 0.0053 valid_accu 0.8363 
企業倫理 | 7    train_loss 0.0020 train_accu 0.9334 valid_loss 0.0053 valid_accu 0.8363 
企業倫理 | 8    train_loss 0.0020 train_accu 0.9334 valid_loss 0.0053 valid_accu 0.8369 < save
企業倫理 | 9    train_loss 0.0020 train_accu 0.9334 valid_loss 0.0053 valid_accu 0.8369 
企業倫理 | 10   train_loss 0.0020 train_accu 0.9334 valid_loss 0.0053 valid_accu 0.8369 
企業倫理 | 11   train_loss 0.0020 train_accu 

In [8]:
## 使用MOE結合標準模型與優化模型
course = pd.read_excel('data/other/course_list.xlsx', 'course')
course = course[course['for_train'] == 1]['course_name'].drop_duplicates()
for c in course:
    c_model = ScorePredict(file_srp='model_pytorch/ScoreRangePredict_MOE_{}.pt'.format(c), srp_requires_grad = False)

    torch.save(c_model, 'model_pytorch/ScorePredict_{}.pt'.format(c))
    criterion = torch.nn.MSELoss(reduction='mean')
    optimizer = torch.optim.Adam(c_model.parameters())
    
    x_train, x_valid, x_test, z_train, z_valid, z_test = getData_ScorePredict(course_name = c, file_model='model_pytorch/ScoreRangePredict_MOE_{}.pt'.format(c))

    train_dl = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, z_train), shuffle=True, batch_size=64) 
    minVL = float(criterion(c_model(x_valid), z_valid))
    minVL_0 = minVL
    minVL_ep = 0

    print('Now processing model:', c)
    print('original VA:', minVL)
    
    for e in range(1000):
        ep = e + 1
        for xb, yb in train_dl:
            pred = c_model(xb)
            loss = criterion(pred, yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        train_loss = float(criterion(c_model(x_train), z_train))
        valid_loss = float(criterion(c_model(x_valid), z_valid))
        test_loss = float(criterion(c_model(x_test), z_test))
        save = ''
        if minVL > valid_loss:
            minVL = valid_loss
            minVL_ep = ep
            save = '< save'
            torch.save(c_model, 'model_pytorch/ScorePredict_{}.pt'.format(c))
        print(c, '|', 
            '{:<4d}'.format(ep), 
            'train_loss', '{:.4f}'.format(train_loss), 
            # 'TA', '{:.4f}'.format(ta), 
            'valid_loss', '{:.4f}'.format(valid_loss), 
            # 'VA', '{:.4f}'.format(va), 
            'test_loss', '{:.4f}'.format(test_loss), 
            save)
        ## early drop
        if ep < 200: ## 至少執行200次
            pass
        elif minVL_ep < ep/2: ## 連續10次小於maxValAcc的一半
            print('Accuracy of validation is CRASH !!')
            break
    print('ReTraining done, maxVA:', minVL_0, '=>', minVL)
    pass

Num of features: 60
Now processing model: 企業倫理
original VA: 7.61794900894165
企業倫理 | 1    train_loss 5.0429 valid_loss 6.0386 test_loss 6.0969 < save
企業倫理 | 2    train_loss 4.7500 valid_loss 6.1623 test_loss 5.0172 
企業倫理 | 3    train_loss 4.2751 valid_loss 5.1255 test_loss 4.6254 < save
企業倫理 | 4    train_loss 3.5248 valid_loss 4.4382 test_loss 4.2354 < save
企業倫理 | 5    train_loss 3.6640 valid_loss 4.4016 test_loss 4.5639 < save
企業倫理 | 6    train_loss 3.1616 valid_loss 4.0132 test_loss 3.9471 < save
企業倫理 | 7    train_loss 3.0894 valid_loss 4.0531 test_loss 3.9847 
企業倫理 | 8    train_loss 3.0846 valid_loss 3.9304 test_loss 4.0218 < save
企業倫理 | 9    train_loss 3.0239 valid_loss 4.2042 test_loss 4.0642 
企業倫理 | 10   train_loss 2.8510 valid_loss 3.9488 test_loss 3.9409 
企業倫理 | 11   train_loss 2.7746 valid_loss 3.8134 test_loss 3.9297 < save
企業倫理 | 12   train_loss 2.8257 valid_loss 3.7617 test_loss 4.0235 < save
企業倫理 | 13   train_loss 2.7813 valid_loss 3.8368 test_loss 4.1635 
企業倫理 | 14   train

In [9]:
model = torch.load('model_pytorch/ScoreRangePredict.pt')
course = pd.read_excel('data/other/course_list.xlsx', 'course')
n_data = pd.read_csv('data/train/normalize.csv')
n_data.set_index('col', inplace=True)

data_week = pd.DataFrame()
for c in course['course_name'].drop_duplicates():
    x_data_raw = pd.read_csv('data/train/ScorePredict_x_{}.csv'.format(c))
    y_data = pd.read_csv('data/train/ScorePredict_y_{}.csv'.format(c))
    
    x_data_raw = x_data_raw[x_data_raw['group'] == 'testing']
    y_data = y_data[y_data['group'] == 'testing']
    y_data.drop(['group'], axis = 1, inplace = True)

    col = ['week']
    y_data_c = x_data_raw[col].drop_duplicates()
    c_model = torch.load('model_pytorch/ScoreRangePredict_MOE_{}.pt'.format(c))
    y_data_c['accuracy_orig'] = y_data_c.apply(lambda c1:accuracy_check(c1, col, x_data_raw, y_data, n_data, model), axis = 1)
    y_data_c['accuracy_opti'] = y_data_c.apply(lambda c1:accuracy_check(c1, col, x_data_raw, y_data, n_data, c_model), axis = 1)
    y_data_c['accuracy_diff'] = y_data_c['accuracy_opti'] - y_data_c['accuracy_orig']
    y_data_c['course_name'] = c
    data_week = pd.concat([data_week, y_data_c])
    pass
data_week.to_excel('data/other/accuracy.xlsx', index = False)